<a href="https://colab.research.google.com/github/Jucicarla/analise-de-sentimento/blob/main/analise_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação de pacotes

test

In [3]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


#Importar pacotes

In [4]:
from textblob import TextBlob

#Análise de Sentimentos

Exemplo de comentário

In [5]:
comentario = "Conheço o restaurante em várias cidades do Brasil. O de Manaus não foge muito da qualidade e beleza. Atendimento bom, ambiente bonito. Mas a comida não surpreendeu, aliás até decepcionou. Não foi aquele sabor que estamos acostumados quando vamos a este restaurante. Pode ter sido um fato isolado, mas."

Texto convertido para o formato TextBlob

In [6]:
comentario_blob = TextBlob(comentario)

Comentário traduzido para Inglês (US)

In [7]:
comentario_blob_us = comentario_blob.translate(to='us')

#Cálculo da subjetividade

O valor da subjetividade é um número real entre [0.0, 1.0], onde 0.0 o texto é indicado como muito objetivo e 1.0 o texto é indicado como muito subjetivo.

In [8]:
for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.subjectivity)

Conheço o restaurante em várias cidades do Brasil. : 0.0
O de Manaus não foge muito da qualidade e beleza. : 1.0
Atendimento bom, ambiente bonito. : 0.8
Mas a comida não surpreendeu, aliás até decepcionou. : 0.625
Não foi aquele sabor que estamos acostumados quando vamos a este restaurante. : 0.0
Pode ter sido um fato isolado, mas. : 0.0


#Cálculo da polaridade

O valor da polaridade é um número real entre os valores [-1.0, 1.0].

In [9]:
for sentence_br, sentence in zip(comentario_blob.sentences, comentario_blob_us.sentences):
    print(sentence_br, ':', sentence.sentiment.polarity)

Conheço o restaurante em várias cidades do Brasil. : 0.0
O de Manaus não foge muito da qualidade e beleza. : -0.05
Atendimento bom, ambiente bonito. : 0.7749999999999999
Mas a comida não surpreendeu, aliás até decepcionou. : -0.55
Não foi aquele sabor que estamos acostumados quando vamos a este restaurante. : 0.0
Pode ter sido um fato isolado, mas. : 0.0
